In [99]:
import pandas as pd

<H1>Load Dataset

In [100]:
#Baca Hasil Scraping
df = pd.read_csv('shopee_reviews.csv')
df.dropna(subset=['content'], inplace=True)

print('Jumlah data', len(df))

Jumlah data 20000


**Insight**
- Sengaja saat scraping hingga 20ribu data agar sesuai dengan penilaian yang minimal 10ribu sample data setelah menghapus beberapa data duplikat

In [101]:
df.head()

,userName,content,score,at
0,Gerald Yoseka,bagus,5,2025-04-08 13:21:45
1,Yuliana Ziraluo,bagus sekali untuk mencari barang yang dibutuhkan,5,2025-04-08 13:21:38
2,Wahyu Qur'an,sebagai member platinum shopee kecewa rasanya ...,1,2025-04-08 13:21:33
3,Aat Atmanah,kadang kesel kalau tokonya yang ngirim sesuai ...,5,2025-04-08 13:19:40
4,Alexander Darren,"belanja enak,,banyak diskon",5,2025-04-08 13:19:36


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  20000 non-null  object
 1   content   20000 non-null  object
 2   score     20000 non-null  int64 
 3   at        20000 non-null  object
dtypes: int64(1), object(3)
memory usage: 625.1+ KB


**Insight**
- Tidak ada missing values pada dataset

In [103]:
duplicate_count = df.duplicated(subset=['content']).sum()
print(f'Jumlah duplicate data: {duplicate_count}')

Jumlah duplicate data: 5254


**Insight**
- ada beberapa duplicate data yang perlu dihapus

In [104]:
df_clean = df.drop_duplicates(subset=['content']).reset_index(drop=True)

print(f'Jumlah data setelah hapus duplicate: {len(df_clean)}')

Jumlah data setelah hapus duplicate: 14746


**Insight**
- Setelah melakukan penghapusan data, maka data yang dapat saya gunakan ada 14746 data. Sehingga sesuai dengan penilaian yang ada

<h1> Text Pre Processing

In [105]:
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Insight**
- Import beberapa library yang dibutuhkan saat text processing saja

In [106]:
#1. Cleaning: Membersihkan teks dari hal-hal tidak perlu
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text) # Menghapus link
    text = re.sub(r"\d+", '', text) # Menghapus angka
    text = re.sub(r"[^\w\s]", '', text) # Menghapus simbol
    text = text.replace('\n', ' ') # Mengganti newline dengan spasi
    text = text.strip()
    return text

#2. Konversi ke huruf kecil
def casefoldingText(text):
    text = text.lower()
    return text

#3. Tokenisasi
def tokenizingText(text):
    tokens = word_tokenize(text)
    return tokens

#4. Menghapus stopwords (Indonesia + Inggris + tambahan kata umum)
def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update([
        'iya','yaa','gak','nya','na','sih','ku',
        'di','ga','ya','gaa','loh','kah','woi','woii','woy',
        'barang', 'belanja',"checkout","co", "yg", "yang", "udah",
        'kalo', 'kalau',"pokoknya", "tolong", "online",
        'spx', 'beli', 'bintang', 'pesan', 'aplikasi', 'apk'
    ])
    return [word for word in text if word not in listStopwords]

#5. Stemming (Bahasa Indonesia)
def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

#6. Ubah list token jadi kalimat lagi
def toSentence(list_words):
    return ' '.join(list_words)

**Insight**
- Membuat beberapa fungsi untuk text processing diantaranya :
    - cleaning text
    - lowercase text (case folding)
    - tokenisasi (Memisahkan kalimat menjadi beberapa text tersendiri)
    - stopwords untuk menghapus beberapa kata yang tidak dibutuhkan
    - stemming
    - list words

In [107]:
slangwords = {
    "bgt": "banget", "maks": "maksimal", "udh": "sudah", "aja": "saja", "blm": "belum", 
    "trs": "terus", "tdk": "tidak", "gk": "tidak", "ga": "tidak", "dr": "dari", "dpt": "dapat",
    "brg": "barang", "bgus": "bagus", "bangus": "bagus", "gpp": "tidak apa", "sgt": "sangat", "bnget" : "banget", "bngtt" : "banget", "baguss" : "bagus",
    "bgtt" : "banget", "bgttt" : "banget", "bgtttt" : "banget", "bgttttt" : "banget", "bgtttttt" : "banget", "bgttttttt" : "banget",
    "jlk": "jelek", "jlkk": "jelek", "jlkkk": "jelek", "jlkkkk": "jelek", "jlkkkkk": "jelek", "jlkkkkkk": "jelek", "jlkkkkkkk": "jelek",
    "lmot": "lemot", "lmott": "lemot", "lmottt": "lemot", "lmotttt": "lemot", "lmottttt": "lemot", "lmotttttt": "lemot", "lmottttttt": "lemot",
    "lmotttttttt": "lemot", "lmottttttttt": "lemot", "lmotttttttttt": "lemot", "lmottttttttttt": "lemot", "lmotttttttttttt": "lemot",
    "blnj": "belanja", "blnjn": "belanja", "blnjnn": "belanja", "blnjnnn": "belanja", "blnjnnnn": "belanja", "blnjnnnnn": "belanja",
    "bngytt": "banget" , "sukaa": "suka", "suka": "suka", "sukaaa": "suka", "sukaaaa": "suka", "sukaaaaa": "suka", "sukaaaaaa": "suka", 
    "ggl": "gagal", "gagl" : "gagal", "gagl": "gagal",
}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []
 
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)
 
    fixed_text = ' '.join(fixed_words)
    return fixed_text

**Insight**
- Membuat slangwords untuk mengganti beberapa kata singkat singkat atau gaul agar bisa menjadi satu kata baku yang bisa di gunakan saat melabelkan nanti

In [108]:
#Membersihkan teks dan menyimpannya di kolom 'text_clean'
df_clean['text_clean'] = df_clean['content'].apply(clean_text)

#Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
df_clean['text_casefoldingText'] = df_clean['text_clean'].apply(casefoldingText)

#Mengganti kata - kata slang dengan kata - kata standar dan menyimpannya di 'text_slangwords'
df_clean['text_slangwords'] = df_clean['text_casefoldingText'].apply(fix_slangwords)

#Memecah teks menjadi token (kata - kata) dan menyimpannya di 'text_tokenizingText'
df_clean['text_tokenizingText'] = df_clean['text_slangwords'].apply(tokenizingText)

#Menghapus kata - kata stop (kata kata umum) dan menyimpannya di 'text_stopwords'
df_clean['text_stopwords'] = df_clean['text_tokenizingText'].apply(filteringText)

#Menghubungkan token - token menjadi kalimat dan menyimpannya di 'text_akhir'
df_clean['text_akhir'] = df_clean['text_stopwords'].apply(toSentence)

**Insight**
- Mengimplementasi semua fungsi yang sudah dibuat tadi ke dalam df_clean dan membuat kolom baru pada table df_clean

In [109]:
df_clean

,userName,content,score,at,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopwords,text_akhir
0,Gerald Yoseka,bagus,5,2025-04-08 13:21:45,bagus,bagus,bagus,[bagus],[bagus],bagus
1,Yuliana Ziraluo,bagus sekali untuk mencari barang yang dibutuhkan,5,2025-04-08 13:21:38,bagus sekali untuk mencari barang yang dibutuhkan,bagus sekali untuk mencari barang yang dibutuhkan,bagus sekali untuk mencari barang yang dibutuhkan,"[bagus, sekali, untuk, mencari, barang, yang, ...","[bagus, mencari, dibutuhkan]",bagus mencari dibutuhkan
2,Wahyu Qur'an,sebagai member platinum shopee kecewa rasanya ...,1,2025-04-08 13:21:33,sebagai member platinum shopee kecewa rasanya ...,sebagai member platinum shopee kecewa rasanya ...,sebagai member platinum shopee kecewa rasanya ...,"[sebagai, member, platinum, shopee, kecewa, ra...","[member, platinum, shopee, kecewa, jnt, hapus,...",member platinum shopee kecewa jnt hapus karna ...
3,Aat Atmanah,kadang kesel kalau tokonya yang ngirim sesuai ...,5,2025-04-08 13:19:40,kadang kesel kalau tokonya yang ngirim sesuai ...,kadang kesel kalau tokonya yang ngirim sesuai ...,kadang kesel kalau tokonya yang ngirim sesuai ...,"[kadang, kesel, kalau, tokonya, yang, ngirim, ...","[kadang, kesel, tokonya, ngirim, sesuai, pesen...",kadang kesel tokonya ngirim sesuai pesenansopi...
4,Alexander Darren,"belanja enak,,banyak diskon",5,2025-04-08 13:19:36,belanja enakbanyak diskon,belanja enakbanyak diskon,belanja enakbanyak diskon,"[belanja, enakbanyak, diskon]","[enakbanyak, diskon]",enakbanyak diskon
...,...,...,...,...,...,...,...,...,...,...
14741,elmayusma neli,enak bnget dpt koin cuman modal rebahan,5,2025-03-25 12:44:00,enak bnget dpt koin cuman modal rebahan,enak bnget dpt koin cuman modal rebahan,enak banget dapat koin cuman modal rebahan,"[enak, banget, dapat, koin, cuman, modal, reba...","[enak, banget, koin, cuman, modal, rebahan]",enak banget koin cuman modal rebahan
14742,Rasya Rasya,sangat cocok untuk belanja online dengan harga...,5,2025-03-25 12:43:01,sangat cocok untuk belanja online dengan harga...,sangat cocok untuk belanja online dengan harga...,sangat cocok untuk belanja online dengan harga...,"[sangat, cocok, untuk, belanja, online, dengan...","[cocok, harga, terjangkau]",cocok harga terjangkau
14743,Grisella,SUMPAH BAGUSS BNGYTT MUDAH BUAT AKU YANG KAUM ...,5,2025-03-25 12:42:32,SUMPAH BAGUSS BNGYTT MUDAH BUAT AKU YANG KAUM ...,sumpah baguss bngytt mudah buat aku yang kaum ...,sumpah bagus banget mudah buat aku yang kaum m...,"[sumpah, bagus, banget, mudah, buat, aku, yang...","[sumpah, bagus, banget, mudah, kaum, mageransu...",sumpah bagus banget mudah kaum mageransuskes s...
14744,Nurhayati Guni,bangus bngtt aplikasi shopee inii😍😍,5,2025-03-25 12:42:29,bangus bngtt aplikasi shopee inii,bangus bngtt aplikasi shopee inii,bagus banget aplikasi shopee inii,"[bagus, banget, aplikasi, shopee, inii]","[bagus, banget, shopee, inii]",bagus banget shopee inii


**Insight**
- Berikut diatas hasil dari penambahan kolom dari fungsi fungsi yang sudah dibuat

<h1> Pelabelan

In [110]:
import requests
import csv
from io import StringIO

# Inisialisasi dictionary kosong untuk menyimpan kata-kata positif
lexicon_positive = dict()

# Mengirim request untuk mengambil data CSV dari GitHub (kamus kata positif)
response_pos = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

# Mengecek apakah request berhasil
if response_pos.status_code == 200:
    # Membaca isi CSV sebagai list baris, dipisahkan berdasarkan koma
    reader = csv.reader(StringIO(response_pos.text), delimiter=',')
    
    # Menyimpan setiap kata positif dan skornya ke dalam dictionary
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])
else:
    # Menampilkan pesan error jika file gagal diambil
    print("Failed to fetch positive lexicon data")


# Inisialisasi dictionary kosong untuk menyimpan kata-kata negatif
lexicon_negative = dict()

# Mengirim request untuk mengambil data CSV dari GitHub (kamus kata negatif)
response_neg = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

# Mengecek apakah request berhasil
if response_neg.status_code == 200:
    # Membaca isi CSV sebagai list baris, dipisahkan berdasarkan koma
    reader = csv.reader(StringIO(response_neg.text), delimiter=',')
    
    # Menyimpan setiap kata negatif dan skornya ke dalam dictionary
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
else:
    # Menampilkan pesan error jika file gagal diambil
    print("Failed to fetch negative lexicon data")


In [111]:
print("bagus" in lexicon_positive)
print("bagus" in lexicon_negative)

if "bagus" in lexicon_positive:
    print("Positif:", lexicon_positive["bagus"])
if "bagus" in lexicon_negative:
    print("Negatif:", lexicon_negative["bagus"])

True
True
Positif: 2
Negatif: -4


**Insight** 
- Sebelum melakukan kode pengecekan kata 'bagus' pada lexicon, saya sudah melakukan pengecekan top words negative dan paling tinggi ialah bagus sehingga tidak normal hasilnya. dan ternyata pada negative nilainya -4 sehingga perlu dilakukan penghapusan

In [112]:
if "bagus" in lexicon_negative:
    del lexicon_negative["bagus"]

**Insight**
- Menghapus kata bagus pada lexicon negative

**Insight**
- Mengambil 2 csv dari github yang berisi kata kata positif dan negatif serta skor dikeduanya yang kemudian dihubungkan kedalam dictionary kosong positif dan negatif dengan nama lexicon_positive dan juga lexicon_negative

In [113]:
# Fungsi untuk menentukan polaritas sentimen dari sebuah teks menggunakan pendekatan lexicon-based
def sentiment_analysis_lexicon_indonesia(text):
    # Inisialisasi skor awal
    score = 0

    # Menambahkan skor positif jika kata ditemukan dalam lexicon positif
    for word in text:
        if word in lexicon_positive:
            score = score + lexicon_positive[word]

    # Menambahkan skor negatif jika kata ditemukan dalam lexicon negatif
    for word in text:
        if word in lexicon_negative:
            score = score + lexicon_negative[word]

    # Menentukan label polaritas berdasarkan nilai skor akhir
    polarity = ''
    if score >= 0:
        polarity = 'positive'  # Jika skor 0 atau lebih, dianggap positif
    elif score < 0:
        polarity = 'negative'  # Jika skor kurang dari 0, dianggap negatif

    # Mengembalikan skor sentimen dan polaritasnya
    return score, polarity


**Insight**
- Membuat fungsi untuk menentukan polaritas sentimen dari text dengan pendekatan lexicon-based dimana diberi fungsi jika menemukan kata positif maka masuk ke dalam lexicon positif jika negative maka masuk kedalam lexicon negative. dan jika nilai score akhir lebih dari sama dengan 0 maka positif, jika kurang dari 0 maka negative

In [114]:
# Menerapkan fungsi analisis sentimen ke setiap baris pada kolom 'text_stopwords'
results = df_clean['text_stopwords'].apply(sentiment_analysis_lexicon_indonesia)

# Mengubah hasil menjadi dua list: satu untuk skor, satu untuk label polaritas
results = list(zip(*results))

# Menyimpan skor polaritas ke kolom baru 'polarity_score'
df_clean['polarity_score'] = results[0]

# Menyimpan label sentimen (positive/negative) ke kolom 'polarity'
df_clean['polarity'] = results[1]

# Menampilkan jumlah data untuk setiap label sentimen
print(df_clean['polarity'].value_counts())


polarity
positive    11419
negative     3327
Name: count, dtype: int64


**Insight**
- Menerapkan fungsi analisis sentimen berbasis lexicon (sentiment_analysis_lexicon_indonesia) pada teks yang telah dibersihkan (text_stopwords).
- Menghasilkan:
    - Skor sentimen numerik (polarity_score)
    - Label kategori sentimen (polarity: seperti 'positive', 'negative', dst)
- Menyimpan hasilnya ke dalam DataFrame (df_clean)
- Menampilkan jumlah data untuk tiap label sentimen → untuk mengecek distribusi kelas.
- Hasil didapatkan positive 9854, dan negative 4892

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X = df_clean['text_akhir']
y = df_clean['polarity']

tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)

features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

features_df

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

**Insight**
- Membuat representasi fitur dari teks menggunakan TF-IDF
- Mengontrol fitur yang digunakan dengan parameter filtering
- Split data menjadi data latih dan data uji (80%:20%)

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Asumsikan 'text_akhir' udah bersih & 'polarity' adalah label sentimen
texts = df_clean['text_akhir']
labels = df_clean['polarity']

# TF-IDF
tfidf = TfidfVectorizer(max_features=1000, stop_words=None)
X_tfidf = tfidf.fit_transform(texts)

# Buat DataFrame TF-IDF
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
tfidf_df['label'] = labels.values

# Rata-rata skor TF-IDF per label
mean_tfidf = tfidf_df.groupby('label').mean().T

# Tampilkan 15 kata paling informatif per kelas
for label in mean_tfidf.columns:
    print(f"\nTop words untuk label '{label}':")
    print(mean_tfidf[label].sort_values(ascending=False).head(15))


Top words untuk label 'negative':
pengiriman    0.075622
shopee        0.046627
banget        0.023598
cepat         0.019472
lemot         0.019419
kurir         0.017504
buruk         0.016357
jelek         0.015978
bagus         0.015962
susah         0.015397
ribet         0.014704
pesanan       0.013902
jt            0.012601
akun          0.012546
iklan         0.012375
Name: negative, dtype: float64

Top words untuk label 'positive':
shopee        0.071492
bagus         0.068116
membantu      0.033025
mantap        0.030032
banget        0.025950
mudah         0.025792
suka          0.024106
puas          0.023748
sesuai        0.023410
murah         0.021243
cepat         0.019641
ongkir        0.016014
berbelanja    0.015387
pelayanan     0.014257
memuaskan     0.014079
Name: positive, dtype: float64


**Insight**
- Menggunakan TF-IDF untuk merepresentasikan teks ke bentuk numerik.
- Menghitung rata-rata skor TF-IDF untuk tiap kata berdasarkan label (positive, negative, [neutral]).
- Menampilkan top words atau kata paling representatif untuk tiap sentimen. 
- Kita bisa melihat beberapa kata yang berpengaruh pada positif dan negative, sehingga saat ada kata kata yang tidak penting dan masuk ke dalam kategori tersebut perlu di hilangkan

In [117]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

#Membuat objek naive bayes
naive_bayes = BernoulliNB()

#Melatih model naive bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)

#Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

#Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

#Menampilkan akurasi
print('Naive Bayes - accuracy_train: ', accuracy_train_nb)
print('Naive Bayes - accuracy_test: ', accuracy_test_nb)

Naive Bayes - accuracy_train:  0.8381654798236691
Naive Bayes - accuracy_test:  0.8386440677966102


**Insight**
- Menggunakan varian BernoulliNB pada naive bayes menghasilkan 0.81 pada akurasi

In [118]:
from sklearn.naive_bayes import MultinomialNB

# Membuat objek model Multinomial Naive Bayes
multinomial_nb = MultinomialNB()

# Melatih model pada data training
multinomial_nb.fit(X_train, y_train)

# Prediksi pada data training dan testing
y_pred_train_mnb = multinomial_nb.predict(X_train)
y_pred_test_mnb = multinomial_nb.predict(X_test)

# Evaluasi akurasi model
accuracy_train_mnb = accuracy_score(y_train, y_pred_train_mnb)
accuracy_test_mnb = accuracy_score(y_test, y_pred_test_mnb)

# Menampilkan hasil akurasi
print("MultinomialNB - accuracy_train:", accuracy_train_mnb)
print("MultinomialNB - accuracy_test:", accuracy_test_mnb)

MultinomialNB - accuracy_train: 0.8683452017633096
MultinomialNB - accuracy_test: 0.868135593220339


**Insight**
- Sudah mencoba mengubah bebera text processing namun yang dihasilkan tadi mentok 0.81 sehingga saya mencoba menggunakan varian lain yang lebih cocok untuk sentimen pada naive bayes yaitu MultinomialNB mendapatkan akurasi 0.84

In [119]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Data
X = df_clean['text_akhir']  # kolom teks hasil preprocessing akhir
y = df_clean['polarity']    # label sentimen: 'positive', 'negative', (atau 'neutral')

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.9)
X_tfidf = tfidf.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Logistic Regression Model
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

# Evaluasi
y_pred_train = logreg.predict(X_train)
y_pred_test = logreg.predict(X_test)

acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)

print("Logistic Regression - Train Accuracy:", round(acc_train, 4))
print("Logistic Regression - Test Accuracy :", round(acc_test, 4))
print("\nClassification Report:\n", classification_report(y_test, y_pred_test))


Logistic Regression - Train Accuracy: 0.9326
Logistic Regression - Test Accuracy : 0.9136

Classification Report:
               precision    recall  f1-score   support

    negative       0.92      0.66      0.77       639
    positive       0.91      0.98      0.95      2311

    accuracy                           0.91      2950
   macro avg       0.92      0.82      0.86      2950
weighted avg       0.91      0.91      0.91      2950



**Insight**
- Alhamdulillah menggunakan logistic jadi 0.93 tertingginya untuk akurasi train

<h1>Simpan Model dan Vectorizer</h1>

In [120]:
import joblib

# Simpan model Logistic Regression
joblib.dump(logreg, 'model_sentimen_logistic.pkl')

# Simpan TF-IDF vectorizer
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

# Testing

In [121]:
# Load model dan vectorizer
logistic_model = joblib.load('model_sentimen_logistic.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')

# GANTI teks ini untuk testing
user_input = "aplikasi ini sangat membantu dan bagus sekali"

# Transformasi input ke TF-IDF
input_vector = tfidf_vectorizer.transform([user_input])

# Prediksi sentimen
predicted_label = logistic_model.predict(input_vector)[0]

# Tampilkan hasil
print(f"Teks: {user_input}")
print(f"Hasil Analisis Sentimen: {predicted_label.upper()}")

# (Opsional) Confidence Score
proba = logistic_model.predict_proba(input_vector)
print(f"Confidence: {max(proba[0]) * 100:.2f}%")

Teks: aplikasi ini sangat membantu dan bagus sekali
Hasil Analisis Sentimen: POSITIVE
Confidence: 99.10%
